## Multi-Agent Systems with LangGraph Tutorial

This notebook demonstrates **multi-agent orchestration** using supervisor patterns. Multi-agent systems decompose complex problems by distributing specialized tasks across different agents.

### Architecture Concepts:
- **Supervisor Agent**: Orchestrates task distribution and coordination
- **Specialized Agents**: Focus on specific domains (research, math, etc.)
- **Agent Delegation**: Intelligent routing of tasks to appropriate experts
- **Collaborative Workflows**: Agents working together on complex problems

### Benefits of Multi-Agent Systems:
- **Specialization**: Each agent excels in its domain
- **Scalability**: Add new agents for new capabilities
- **Maintainability**: Isolated agent logic is easier to debug and improve
- **Parallel Processing**: Multiple agents can work simultaneously

## Environment Setup for Multi-Agent Systems

Install specialized packages for multi-agent orchestration:
- **langgraph-supervisor**: Framework for creating supervisor-agent hierarchies
- **create_supervisor & create_react_agent**: Pre-built patterns for agent management

In [ ]:
# !pip install langgraph-supervisor langchain-openai

In [ ]:
# !pip install langchain langchain_community langgraph

## Central LLM Configuration

Set up the shared language model that will power all agents in the system. Using a single LLM instance ensures consistency in reasoning across different agents while maintaining cost efficiency.

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

# Initialize OpenAI model
model = ChatOpenAI(temperature=0.3, model="gpt-4.1-nano")


## Specialized Tool Creation

Create domain-specific tools that will be assigned to different agents:

- **Search tools**: For research agents requiring web access
- **Mathematical tools**: For computation-focused agents
- **Tool separation**: Each agent gets tools relevant to its specialization
- **Clear boundaries**: Prevents agents from overstepping their intended domains

This separation of concerns makes agents more reliable and predictable.

In [ ]:
# !pip install -q duckduckgo-search <--- deprecated
# !pip install -q ddgs

In [5]:
# from langchain_community.tools import DuckDuckGoSearchRun

# def search_duckduckgo(query: str):
#     """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
#     search = DuckDuckGoSearchRun()
#     return search.invoke(query)

from ddgs import DDGS

def search_duckduckgo(query: str):
    """Searches DuckDuckGo using the ddgs library."""
    results = DDGS().text(query, max_results=5)
    return ' '.join([f"{result['title']} {result['body']}" for result in results])

# Example usage
# result = search_duckduckgo("what are AI agent")
# print(result)

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

## Math Expert Agent Creation

Create a specialized agent focused on mathematical computations:

- **Limited tool set**: Only mathematical operations (add, multiply)
- **Specific prompt**: Guides the agent to focus on math problems exclusively
- **Single tool usage**: "Always use one tool at a time" prevents complex chaining
- **Domain expertise**: Agent becomes highly reliable for computational tasks

Specialization leads to better performance and fewer errors in specific domains.

In [6]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

## Research Expert Agent Creation

Create a specialized agent focused on information gathering and research:

- **Search capabilities**: Access to web search through DuckDuckGo
- **Research prompt**: Emphasizes research expertise and information gathering
- **Math restriction**: "Do not do any math" maintains clear boundaries
- **Information specialization**: Agent becomes expert at finding and synthesizing information

This separation ensures each agent excels in its specific domain without overlap.

In [7]:
research_agent = create_react_agent(
    model=model,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)


## Supervisor Workflow Creation

The **supervisor agent** orchestrates the multi-agent system:

- **Agent management**: Oversees both research and math agents
- **Task routing**: Analyzes user queries to determine which agent should handle the task
- **Delegation logic**: Routes current events to research agent, math problems to math agent
- **Coordination**: Ensures smooth collaboration between specialized agents

The supervisor embodies the system's intelligence for task distribution and workflow management.

In [8]:
# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

## Testing Research Agent Specialization

Test the multi-agent system with a research-focused query. The supervisor should:
1. **Analyze the query**: "What is quantum computing?" requires research expertise
2. **Route to research agent**: Delegate the task to the specialized research agent
3. **Execute search**: Research agent uses web search to gather information
4. **Return results**: Provide comprehensive information about quantum computing

In [9]:
# Compile and run
app = workflow.compile()

result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is quantum computing?"
        }
    ]
})

In [11]:
result

{'messages': [HumanMessage(content='what is quantum computing?', additional_kwargs={}, response_metadata={}, id='3222ace7-abc1-4ee4-9bf3-37ebec24c85f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2oX5TwIbvT6YOxNdIbgXlRuT', 'function': {'arguments': '{}', 'name': 'transfer_to_research_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 101, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C9jc4jgqfgI3IlBaoSvCtU30o7nxI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--0131eb5c-cbbb-4074-9df9-7a72f0f599f6-0', tool_calls=[{'name': 'transfer_to_research_expert', 'a

In [13]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is quantum computing?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_2oX5TwIbvT6YOxNdIbgXlRuT)
 Call ID: call_2oX5TwIbvT6YOxNdIbgXlRuT
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Quantum computing is a type of computation that leverages the principles of quantum mechanics to perform operations on data. Unlike classical computers, which use bits as the smallest unit of information (either 0 or 1), quantum computers use quantum bits or qubits. Qubits can exist in multiple states simultaneously thanks to a property called superposition, and they can be entangled with each other, all

## Testing Multi-Agent Coordination

Test with a complex query requiring both research and mathematical capabilities:
1. **Research phase**: Get weather information for Delhi (research agent)
2. **Mathematical phase**: Multiply by 2 and add 5 (math agent)  
3. **Supervisor coordination**: Route tasks to appropriate agents and combine results
4. **Seamless workflow**: User gets complete solution without needing to understand the internal agent structure

This demonstrates the power of multi-agent systems for complex, multi-step problems.

In [14]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is the weather in Galle, Sri Lanka today?. Multiply it by 2 and add 5"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is the weather in Galle, Sri Lanka today?. Multiply it by 2 and add 5
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_NcfZKhL8pTAftOZoJsBIEALd)
 Call ID: call_NcfZKhL8pTAftOZoJsBIEALd
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

The current weather in Galle, Sri Lanka, is overcast with a temperature of approximately 27.8°C. 

Since you asked to multiply this by 2 and add 5, I will perform that calculation now.
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervi